In [1]:
from typing import List, Union, Callable, Tuple
from functools import partial
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', None, 'b', 'b', 'a', None],
                  'key2': pd.Series([1, 2, 1, 2, 1, None, 1], dtype='Int64'),
                  'data1': np.random.standard_normal(7),
                  'data2': np.random.standard_normal(7)})

In [3]:
df

,key1,key2,data1,data2
0,a,1,0.979560,-0.190768
1,a,2,0.116407,-1.898274
2,None,1,1.199956,0.376132
3,b,2,1.432026,1.696011
4,b,1,-0.464025,-1.627408
5,a,<NA>,0.042425,0.949409
6,None,1,0.070341,0.854331


In [4]:
grouped = df.groupby('key1')

In [5]:
grouped

In [6]:
grouped['data1'].nsmallest(2)

key1   
a     5    0.042425
      1    0.116407
b     4   -0.464025
      3    1.432026
Name: data1, dtype: float64

In [7]:
#to use one's own aggregate function: pass any function aggregating any array to the aggregate method (alias agg).
grouped['data1'].nsmallest(2)

key1   
a     5    0.042425
      1    0.116407
b     4   -0.464025
      3    1.432026
Name: data1, dtype: float64

In [8]:
grouped['data1'].nlargest(2)

key1   
a     0    0.979560
      1    0.116407
b     3    1.432026
      4   -0.464025
Name: data1, dtype: float64

In [9]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [10]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.937134,2.847683
b,1,1.896051,3.323420


In [11]:
#you may notice that some methods also work even if they are not aggregations
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.379464  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.484000  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.547983  0.979560   3.0 -0.379878  1.433229 -1.898274 -1.044521   
b     0.958013  1.432026   2.0  0.034301  2.350013 -1.627408 -0.796553   

                                    
           50%       75%       max  
key1                                
a    -0.190768  0.379321  0.949409  
b     0.034301  0.865156  1.696011  

[2 rows x 24 columns]

In [12]:
#column-wise and multiple function application
#case study: Tips dataset
tips = pd.read_csv('examples/tips.csv')

In [13]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [14]:
#now add a tip_pct column
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [15]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [16]:
#group tips by "day" and "smoker"
grouped = tips.groupby(['day', 'smoker'])

In [17]:
#note for descriptive statistics like in table 10-1, we can pass the function as a string
grouped_pct = grouped['tip_pct']

In [18]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [19]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [20]:
#To implement a list of functions or function names, then we get back a DataFrame with column
#names taken from the functions
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [21]:
#To specify names for functions: e.g. with lambda functions: pass a list of tuples.
#Item 0 becomes the name.
#Item 1 becomes the description.
grouped_pct.agg([('average', 'mean'), ('stdev', np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [22]:
#DataFrames
#We can apply the same function to all of the columns or different functions per column.
#Example: Let's compute three statistics the tip_pct and total_bill columns: 1) count 2) mean 3) max.
functions : List[Union[str, Callable]] = ['count', 'mean', 'max']

In [23]:
result = grouped[['tip_pct', 'total_bill']].agg(functions)

In [24]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [25]:
#resulting dataframe uses heirarchical columns, just as if we aggregated separately and grouped using
#the column names as the keys argument
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [26]:
#we can also pass a list of tuples with custom names
ftuples : List[Tuple[str, Union[str, Callable]]] = [('Average', 'mean'), ('Variance', np.var)]
ftuples

[('Average', 'mean'),
 ('Variance',
  <function numpy.var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>, *, where=<no value>)>)]

In [27]:
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [28]:
grouped[['size', 'total_bill']].agg(ftuples)

size           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      2.250000  0.250000  18.420000   25.596333
     Yes     2.066667  0.352381  16.813333   82.562438
Sat  No      2.555556  0.616162  19.661778   79.908965
     Yes     2.476190  0.743322  21.276667  101.387535
Sun  No      2.929825  1.066416  20.506667   66.099980
     Yes     2.578947  0.812865  24.120000  109.046044
Thur No      2.488889  1.391919  17.113111   59.625081
     Yes     2.352941  0.492647  19.190588   69.808518

In [29]:
#Now, what if we wanted to pass different functions to each column. Pass a dictionary to
#agg that contains a mapping of column names to any of the function specifications we have used.
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [30]:
#multiple functions applied to one column, single functions applied to the others
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})
#we have heirarchical columns because multiple functions are applied to one column

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [31]:
#returning aggregated data without row indexes
#Until now, we've included the data with an index composed of group-key combinations. 
#if we use the as_index=False argument, we can use a regular numerical index
tips.groupby(['day', 'smoker'], as_index=False).mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [35]:
#apply split combine
#df.apply and series.apply are the most general-purpose groupby commands
#apply applies the function to each element and attempts to concatenate them
#Example: We want to extract the top five columns by group
def top(df: pd.DataFrame, n: int, column: str):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6, column='tip_pct')

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [36]:
#if we group by smoker and call apply with this function here is the result
tips.groupby('smoker').apply(top, n=6, column='tip_pct')
#1) tips function called on each group
#2) 

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
       6          8.77  2.00     No   Sun  Dinner     2  0.228050
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       93        16.32  4.30    Yes   Fri  Dinner     2  0.263480

In [39]:
top_6_tip_pct = partial(top, n=6, column='tip_pct')
tips.groupby('smoker').apply(top_6_tip_pct)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
       6          8.77  2.00     No   Sun  Dinner     2  0.228050
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       93        16.32  4.30    Yes   Fri  Dinner     2  0.263480

In [40]:
#if we pass a function that takes keywords to apply, we can pass these after the function
#or use partial functions as above (TS: partial functions above not included in book)
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [ ]:
#apply examples